In [8]:
# __Run this cell to update AQI numbers__

import requests # version == '2.21.0'
import pandas as pd # version == '0.23.4'
import numpy as np # # version == '1.15.4'
from datetime import datetime
import concurrent.futures

pd.set_option("display.max_rows", None)

# read in zip codes dataframe
zips = pd.read_excel('zipcodes_updated.xlsx', names=['zipcode', 'other'])

# function to find state in one text string
def get_state(row):
    try:
        return row[1]
    except:
        return 'not a state'
    
# apply above function
zips['state'] = zips.other.str.split('>>').apply(get_state)

# function to find city in one text string
def get_cities(row):
    try:
        return row[2]
    except:
        return 'not a city'

# apply above function    
zips['cities'] = zips.other.str.split('>>').apply(get_cities)

# clean state and city text
zips['state'] = zips.state.str.strip(" '")
zips['state'] = zips.state.str.strip()
zips['cities'] = zips.cities.str.strip(']')
zips['cities'] = zips.cities.str.strip("'")
zips['cities'] = zips.cities.str.strip()
zips['cities'] = zips.cities.str.strip("''")

# combine city and state to merge later/deal with duplicate city names
zips['city-state'] = zips['cities'] + ', ' + zips['state']

# function to get all cities/regions and their corresponding AQI value    
def getCityValues(link):
    
    req = requests.get(link) # request AirNow link
    df =  pd.read_html(req.text)[7] # get dataframe of AQI numbers
    state = pd.read_html(req.text)[0].loc[0, 2] # get state
    states = [state for n in range(df.shape[0])] # enumerate states to add to df
    
    # find indexes that correspond to cities in dataframe 
    def is_string(x):
        try:
            if int(x):
                return False
        except:
            return True
        else:
            return True
        
    # apply above function  
    indexes = df[1].dropna()[df[1].dropna().apply(is_string)].index.tolist()
    
    df = df.iloc[indexes] # use above index to find cities in html
    df['state'] = pd.Series(states) # make series with state list
    df['city-state'] = df[1] + ', ' + df['state'] # combine city and state to handle duplicate city names
    
    # function to get all AQI values from table
    def find_numbers(lst):
        numbers = []
        for x in lst:
            try:
                if int(x):
                    numbers.append(int(x))
            except:
                pass
            else:
                pass
        return numbers
    
    # function that counts AQI values per city
    def count_numbers(lst):
        return len(lst)
    
    df['numbers'] = df[0].str.split(' ').apply(find_numbers)
    df['number_count'] = df.numbers.apply(count_numbers)
    
    # function that gets the correct (current) AQI value from html
    def getAQIValues(row):

        if row['number_count'] == 0:
            return np.nan

        elif row['number_count'] == 3:
            return row['numbers'][-1]

        elif row['number_count'] == 2 and str(row[10]) == 'nan':
            return np.nan

        elif row['number_count'] == 1 and str(row[9]) != 'nan':
            return row['numbers'][0]

        elif row['number_count'] == 1 and str(row[9]) == 'nan':
            return np.nan

        elif row['number_count'] == 2 and str(row[10]) != 'nan':
            return row['numbers'][1]

        else:
            return np.nan

    df['values'] = df.T.apply(getAQIValues) # apply getAQIValues function
    small_df = df[[1, 'values', 'city-state']] # select dataframe columns
    small_df = small_df.rename({1: 'city'}, axis=1) # change column name

    small_df['date'] = datetime.now() # make column with current datetime

    return small_df # return state dataframe

with concurrent.futures.ThreadPoolExecutor() as executor:
    # for loop to get all state links
    links = []
    for n in range(1, 53):
        link = 'https://www.airnow.gov/index.cfm?action=airnow.local_state&stateid=' + str(n)
        links.append(link)
    results = executor.map(getCityValues, links)

    # apply getCityValues function to all state links and append them to a list
    dfs = []
    for result in results:
        dfs.append(result)

final = pd.concat(dfs) # concatenate all state dataframes

final = pd.merge(zips, final, on='city-state', how='inner') # merge zip codes and aqi data frames

Wall time: 2min 13s


In [9]:
# __Run this cell to filter by AQI value and display results__

zip_df = final.copy() # copy final dataframe

zip_df = zip_df[zip_df['values'] >= 100] # Set AQI threshold
zip_df = zip_df[['zipcode', 'state', 'city', 'values', 'date']] # make specific columns
zip_df

,zipcode,state,city,values,date
13903,04471,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13904,04497,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13905,04730,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13906,04732,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13907,04733,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13908,04734,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13909,04735,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13910,04736,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13911,04737,Maine,Northern,176.0,2020-01-31 16:25:36.910360
13912,04738,Maine,Northern,176.0,2020-01-31 16:25:36.910360


In [5]:
# __Run this cell to save the air quality data in table above to a csv__
# __Don't forget to change the number in parentheses if saving multiple files in one day

date = str(pd.to_datetime(datetime.now())).replace(' ', '')[:10] # current date
zip_df.reset_index().drop(columns=['index']).to_csv('AQI' + date + '(2)' + '.csv') # file format